In [1]:
#dependencies
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup 
from splinter import Browser
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/ppp_geo_fips23.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5502308207,AK,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309,2.020002e+13
1,6110847106,AK,"7000 Uula St, BARROW, AK",71.297290,-156.728897,2.185000e+13
2,4539098204,AK,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718,2.020002e+13
3,5120868804,AK,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283,2.020003e+13
4,6650277102,AK,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793,2.122001e+13


In [ ]:
pd.set_option('display.max_columns', None)
#Latitude, Longitude included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

In [ ]:
fips_check_scraped_null_il = fips_check_scraped_null.loc[fips_check_scraped_null["BorrowerState"]=="IL"]
fips_check_scraped_null_il

Manually Updated File Read-IN - convert xlsx to csv

In [103]:
data_xls = pd.read_excel('../../../data/null/updated/not_avail_fips_update.xlsx', dtype=str, index_col=None)
data_xls.to_csv('../../../data/null/updated/not_avail_fips_update.csv', encoding='utf-8', index=False)
not_avail_update_df = pd.read_csv("../../../data/null/updated/not_avail_fips_update.csv")
not_avail_update_df.index = not_avail_update_df.index.astype('int64')
not_avail_update_df

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,9662437702,MI,"20101 Cornillie Dr, Roseville, MI",42.521542,-82.913466,NaN
1,9774337701,OH,"1255 Norton Rd, Hudson, OH",41.203947,-81.450338,NaN
2,9722187702,OH,"8379 Harrison Pike, Cleves, OH",39.219563,-84.714039,NaN
3,9725917702,MI,"128 E Front St, Traverse City, MI",44.985866,-85.603075,NaN
4,9666867710,OH,"11201 Shaker Blvd #126, Cleveland, OH",41.484647,-81.608183,NaN
5,9662697710,MI,"56409 Precision Dr, New Baltimore, MI",42.717196,-82.808277,NaN
6,9724667708,FL,"2707 Sadler Rd, Fernandina Beach, FL",30.639685,-81.438653,NaN
7,9672967701,OH,"1602 N Curtice Rd, Curtice, OH",41.623289,-83.370857,NaN


<h4>FIPS Script

In [104]:
lat = not_avail_update_df["Lat"].astype(float).astype(str).to_list()
long = not_avail_update_df["Long"].astype(float).astype(str).to_list()

In [105]:
#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [106]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=True)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            #startswith = [s for s in result_digits if s.startswith('51')] #VA
            fips_elements =[element for element in result_digits if len(element)==15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/8 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

Length Check

In [107]:
geo_id_list =geo_id 
print(len(geo_id_list))
print(geo_id_list)

8
[['260992551002000'], ['391535340002028'], ['390610260024000'], ['260899702012011'], ['390351986004000'], ['260992215001001'], ['120890501041005'], ['390950097001024']]


Matching FIPS to DataFrame

In [108]:
fips_fill = not_avail_update_df

In [109]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,9662437702,MI,"20101 Cornillie Dr, Roseville, MI",42.521542,-82.913466,[260992551002000]
1,9774337701,OH,"1255 Norton Rd, Hudson, OH",41.203947,-81.450338,[391535340002028]
2,9722187702,OH,"8379 Harrison Pike, Cleves, OH",39.219563,-84.714039,[390610260024000]
3,9725917702,MI,"128 E Front St, Traverse City, MI",44.985866,-85.603075,[260899702012011]
4,9666867710,OH,"11201 Shaker Blvd #126, Cleveland, OH",41.484647,-81.608183,[390351986004000]
5,9662697710,MI,"56409 Precision Dr, New Baltimore, MI",42.717196,-82.808277,[260992215001001]
6,9724667708,FL,"2707 Sadler Rd, Fernandina Beach, FL",30.639685,-81.438653,[120890501041005]
7,9672967701,OH,"1602 N Curtice Rd, Curtice, OH",41.623289,-83.370857,[390950097001024]


Remove brackets

In [110]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,9662437702,MI,"20101 Cornillie Dr, Roseville, MI",42.521542,-82.913466,260992551002000
1,9774337701,OH,"1255 Norton Rd, Hudson, OH",41.203947,-81.450338,391535340002028
2,9722187702,OH,"8379 Harrison Pike, Cleves, OH",39.219563,-84.714039,390610260024000
3,9725917702,MI,"128 E Front St, Traverse City, MI",44.985866,-85.603075,260899702012011
4,9666867710,OH,"11201 Shaker Blvd #126, Cleveland, OH",41.484647,-81.608183,390351986004000
5,9662697710,MI,"56409 Precision Dr, New Baltimore, MI",42.717196,-82.808277,260992215001001
6,9724667708,FL,"2707 Sadler Rd, Fernandina Beach, FL",30.639685,-81.438653,120890501041005
7,9672967701,OH,"1602 N Curtice Rd, Curtice, OH",41.623289,-83.370857,390950097001024


Check

In [111]:
fips_fill.columns.isnull()

array([False, False, False, False, False, False])

Read in base data

In [112]:
#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/ppp_geo_fips23.csv", index_col=0, dtype={'FIPS_z': str}) 
fips_check['FIPS_z'] = fips_check['FIPS_z'].astype('string').str.split('.').str[0] #format 
fips_check['FIPS_z'] = fips_check['FIPS_z'].str.zfill(15)
fips_check

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5502308207,AK,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309,020200019003022
1,6110847106,AK,"7000 Uula St, BARROW, AK",71.297290,-156.728897,021850001001069
2,4539098204,AK,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718,020200023011009
3,5120868804,AK,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283,020200025012000
4,6650277102,AK,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793,021220010002001
...,...,...,...,...,...,...
965547,9725917702,Not Available,"17 S High St., COLUMBUS, OH",39.961717,-83.001011,NaN
965548,9666867710,Not Available,"17 S High St., COLUMBUS, OH",39.961717,-83.001011,NaN
965549,9662697710,Not Available,"17 S High St., COLUMBUS, OH",39.961717,-83.001011,NaN
965550,9724667708,Not Available,"17 S High St., COLUMBUS, OH",39.961717,-83.001011,NaN


In [114]:
fips_check.loc[fips_check["LoanNumber"]==	9662697710	]

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
965549,9662697710,Not Available,"17 S High St., COLUMBUS, OH",39.961717,-83.001011,<NA>


Update

In [115]:
fips_check.loc[fips_check.LoanNumber.isin(fips_fill.LoanNumber), ['BorrowerState', 'full_add', 'Lat', 'Long', 'FIPS_z']] = fips_fill[['BorrowerState', 'full_add', 'Lat', 'Long', 'FIPS_z']].values
fips_check

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5502308207,AK,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309,020200019003022
1,6110847106,AK,"7000 Uula St, BARROW, AK",71.297290,-156.728897,021850001001069
2,4539098204,AK,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718,020200023011009
3,5120868804,AK,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283,020200025012000
4,6650277102,AK,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793,021220010002001
...,...,...,...,...,...,...
965547,9725917702,MI,"128 E Front St, Traverse City, MI",44.985866,-85.603075,260899702012011
965548,9666867710,OH,"11201 Shaker Blvd #126, Cleveland, OH",41.484647,-81.608183,390351986004000
965549,9662697710,MI,"56409 Precision Dr, New Baltimore, MI",42.717196,-82.808277,260992215001001
965550,9724667708,FL,"2707 Sadler Rd, Fernandina Beach, FL",30.639685,-81.438653,120890501041005


Verify

In [116]:
fips_check.loc[fips_check["LoanNumber"]==	9662697710	]

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
965549,9662697710,MI,"56409 Precision Dr, New Baltimore, MI",42.717196,-82.808277,260992215001001


Write to MainFrame

In [117]:
fips_check.to_csv("../../../data/state_data/geo/ppp_geo_fips23.csv") 

Run through fipssplit after write to update state Files

<b>Check State Codes match for FIPS_z :2<br>

In [118]:
ak_ch = fips_check.loc[(fips_check["BorrowerState"] == 'AK') & (fips_check["FIPS_z"].str[:2] != '02')]
al_ch = fips_check.loc[(fips_check["BorrowerState"] == 'AL') & (fips_check["FIPS_z"].str[:2] != '01')]
ar_ch = fips_check.loc[(fips_check["BorrowerState"] == 'AR') & (fips_check["FIPS_z"].str[:2] != '05')]
as_ch = fips_check.loc[(fips_check["BorrowerState"] == 'AS') & (fips_check["FIPS_z"].str[:2] != '60')]
az_ch = fips_check.loc[(fips_check["BorrowerState"] == 'AZ') & (fips_check["FIPS_z"].str[:2] != '04')]

ca_ch = fips_check.loc[(fips_check["BorrowerState"] == 'CA') & (fips_check["FIPS_z"].str[:2] != '06')]
co_ch = fips_check.loc[(fips_check["BorrowerState"] == 'CO') & (fips_check["FIPS_z"].str[:2] != '08')]
ct_ch = fips_check.loc[(fips_check["BorrowerState"] == 'CT') & (fips_check["FIPS_z"].str[:2] != '09')]
dc_ch = fips_check.loc[(fips_check["BorrowerState"] == 'DC') & (fips_check["FIPS_z"].str[:2] != '11')]
de_ch = fips_check.loc[(fips_check["BorrowerState"] == 'DE') & (fips_check["FIPS_z"].str[:2] != '10')]

fl_ch = fips_check.loc[(fips_check["BorrowerState"] == 'FL') & (fips_check["FIPS_z"].str[:2] != '12')]
ga_ch = fips_check.loc[(fips_check["BorrowerState"] == 'GA') & (fips_check["FIPS_z"].str[:2] != '13')]
gu_ch = fips_check.loc[(fips_check["BorrowerState"] == 'GU') & (fips_check["FIPS_z"].str[:2] != '66')]
hi_ch = fips_check.loc[(fips_check["BorrowerState"] == 'HI') & (fips_check["FIPS_z"].str[:2] != '15')]
ia_ch = fips_check.loc[(fips_check["BorrowerState"] == 'IA') & (fips_check["FIPS_z"].str[:2] != '19')]

id_ch = fips_check.loc[(fips_check["BorrowerState"] == 'ID') & (fips_check["FIPS_z"].str[:2] != '16')]
il_ch = fips_check.loc[(fips_check["BorrowerState"] == 'IL') & (fips_check["FIPS_z"].str[:2] != '17')]
in_ch = fips_check.loc[(fips_check["BorrowerState"] == 'IN') & (fips_check["FIPS_z"].str[:2] != '18')]
ks_ch = fips_check.loc[(fips_check["BorrowerState"] == 'KS') & (fips_check["FIPS_z"].str[:2] != '20')]
ky_ch = fips_check.loc[(fips_check["BorrowerState"] == 'KY') & (fips_check["FIPS_z"].str[:2] != '21')]

la_ch = fips_check.loc[(fips_check["BorrowerState"] == 'LA') & (fips_check["FIPS_z"].str[:2] != '22')]
ma_ch = fips_check.loc[(fips_check["BorrowerState"] == 'MA') & (fips_check["FIPS_z"].str[:2] != '25')]
md_ch = fips_check.loc[(fips_check["BorrowerState"] == 'MD') & (fips_check["FIPS_z"].str[:2] != '24')]
me_ch = fips_check.loc[(fips_check["BorrowerState"] == 'ME') & (fips_check["FIPS_z"].str[:2] != '23')]
mi_ch = fips_check.loc[(fips_check["BorrowerState"] == 'MI') & (fips_check["FIPS_z"].str[:2] != '26')]

mn_ch = fips_check.loc[(fips_check["BorrowerState"] == 'MN') & (fips_check["FIPS_z"].str[:2] != '27')]
mo_ch = fips_check.loc[(fips_check["BorrowerState"] == 'MO') & (fips_check["FIPS_z"].str[:2] != '29')]
mp_ch = fips_check.loc[(fips_check["BorrowerState"] == 'MP') & (fips_check["FIPS_z"].str[:2] != '69')]
ms_ch = fips_check.loc[(fips_check["BorrowerState"] == 'MS') & (fips_check["FIPS_z"].str[:2] != '28')]
mt_ch = fips_check.loc[(fips_check["BorrowerState"] == 'MT') & (fips_check["FIPS_z"].str[:2] != '30')]

nc_ch = fips_check.loc[(fips_check["BorrowerState"] == 'NC') & (fips_check["FIPS_z"].str[:2] != '37')]
nd_ch = fips_check.loc[(fips_check["BorrowerState"] == 'ND') & (fips_check["FIPS_z"].str[:2] != '38')]
ne_ch = fips_check.loc[(fips_check["BorrowerState"] == 'NE') & (fips_check["FIPS_z"].str[:2] != '31')]
nh_ch = fips_check.loc[(fips_check["BorrowerState"] == 'NH') & (fips_check["FIPS_z"].str[:2] != '33')]
nj_ch = fips_check.loc[(fips_check["BorrowerState"] == 'NJ') & (fips_check["FIPS_z"].str[:2] != '34')]

nm_ch = fips_check.loc[(fips_check["BorrowerState"] == 'NM') & (fips_check["FIPS_z"].str[:2] != '35')]
nv_ch = fips_check.loc[(fips_check["BorrowerState"] == 'NV') & (fips_check["FIPS_z"].str[:2] != '32')]
ny_ch = fips_check.loc[(fips_check["BorrowerState"] == 'NY') & (fips_check["FIPS_z"].str[:2] != '36')]
oh_ch = fips_check.loc[(fips_check["BorrowerState"] == 'OH') & (fips_check["FIPS_z"].str[:2] != '39')]
ok_ch = fips_check.loc[(fips_check["BorrowerState"] == 'OK') & (fips_check["FIPS_z"].str[:2] != '40')]

or_ch = fips_check.loc[(fips_check["BorrowerState"] == 'OR') & (fips_check["FIPS_z"].str[:2] != '41')]
pa_ch = fips_check.loc[(fips_check["BorrowerState"] == 'PA') & (fips_check["FIPS_z"].str[:2] != '42')]
pr_ch = fips_check.loc[(fips_check["BorrowerState"] == 'PR') & (fips_check["FIPS_z"].str[:2] != '72')]
ri_ch = fips_check.loc[(fips_check["BorrowerState"] == 'RI') & (fips_check["FIPS_z"].str[:2] != '44')]
sc_ch = fips_check.loc[(fips_check["BorrowerState"] == 'SC') & (fips_check["FIPS_z"].str[:2] != '45')]

sd_ch = fips_check.loc[(fips_check["BorrowerState"] == 'SD') & (fips_check["FIPS_z"].str[:2] != '46')]
tn_ch = fips_check.loc[(fips_check["BorrowerState"] == 'TN') & (fips_check["FIPS_z"].str[:2] != '47')]
tx_ch = fips_check.loc[(fips_check["BorrowerState"] == 'TX') & (fips_check["FIPS_z"].str[:2] != '48')]
ut_ch = fips_check.loc[(fips_check["BorrowerState"] == 'UT') & (fips_check["FIPS_z"].str[:2] != '49')]
va_ch = fips_check.loc[(fips_check["BorrowerState"] == 'VA') & (fips_check["FIPS_z"].str[:2] != '51')]

vi_ch = fips_check.loc[(fips_check["BorrowerState"] == 'VI') & (fips_check["FIPS_z"].str[:2] != '78')]
vt_ch = fips_check.loc[(fips_check["BorrowerState"] == 'VT') & (fips_check["FIPS_z"].str[:2] != '50')]
wa_ch = fips_check.loc[(fips_check["BorrowerState"] == 'WA') & (fips_check["FIPS_z"].str[:2] != '53')]
wi_ch = fips_check.loc[(fips_check["BorrowerState"] == 'WI') & (fips_check["FIPS_z"].str[:2] != '55')]
wv_ch = fips_check.loc[(fips_check["BorrowerState"] == 'WV') & (fips_check["FIPS_z"].str[:2] != '54')]

wy_ch = fips_check.loc[(fips_check["BorrowerState"] == 'WY') & (fips_check["FIPS_z"].str[:2] != '56')]



frames =[ak_ch,al_ch,ar_ch,as_ch,az_ch,
ca_ch,co_ch,ct_ch,dc_ch,de_ch,
fl_ch,ga_ch,gu_ch,hi_ch,ia_ch,
id_ch,il_ch,in_ch,ks_ch,ky_ch,
la_ch,ma_ch,md_ch,me_ch,mi_ch,
mn_ch,mo_ch,mp_ch,ms_ch,mt_ch,
nc_ch,nd_ch,ne_ch,nh_ch,nj_ch,
nm_ch,nv_ch,ny_ch,oh_ch,ok_ch,
or_ch,pa_ch,pr_ch,ri_ch,sc_ch,
sd_ch,tn_ch,tx_ch,ut_ch,va_ch,
vi_ch,vt_ch,wa_ch,wi_ch,wv_ch,
wy_ch]

result = pd.concat(frames,ignore_index=True)
result


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z


Adjust for Correct Borrower State / Address for Correct FIPS

In [119]:
'''
fips_check.loc[fips_check["LoanNumber"] == 3506697210, 'BorrowerState'] = 'IN'
fips_check.loc[fips_check["LoanNumber"] == 	1919668403, 'Lat'] = 45.939522707297485
fips_check.loc[fips_check["LoanNumber"] == 	1919668403, 'Long'] = -86.28201139556197
fips_check.loc[fips_check["LoanNumber"] == 	1919668403, 'FIPS_z'] = '261530002004003'
fips_check.loc[fips_check["LoanNumber"] == 	2703197109, 'Lat'] = 45.939522707297485
fips_check.loc[fips_check["LoanNumber"] == 	2703197109, 'Long'] = -86.28201139556197
fips_check.loc[fips_check["LoanNumber"] == 	2703197109, 'FIPS_z'] = '261530002004003'
fips_check.loc[fips_check["LoanNumber"] ==  4357417000, 'FIPS_z'] = '170850202004000'
fips_check.loc[fips_check["LoanNumber"] == 	4139637205, 'Lat'] = 32.99734644174747
fips_check.loc[fips_check["LoanNumber"] == 	4139637205, 'Long'] = -93.46100681895895
fips_check.loc[fips_check["LoanNumber"] == 	4139637205, 'FIPS_z'] = '221190313003012'
fips_check.loc[fips_check["LoanNumber"] == 3451078504, 'BorrowerState'] = 'FL'
fips_check.loc[fips_check["LoanNumber"] == 3451078504, 'full_add'] = '150 N Nova Rd, Daytona Beach, FL'
fips_check.loc[fips_check["LoanNumber"] == 3451078504, 'Lat'] = '29.2050623399283'
fips_check.loc[fips_check["LoanNumber"] == 3451078504, 'Long'] = '-81.04197929024815'
fips_check.loc[fips_check["LoanNumber"] == 3451078504, 'FIPS_z'] = '121270818003011'
fips_check.loc[fips_check["LoanNumber"] == 5657827001, 'BorrowerState'] = 'ND'
fips_check.loc[fips_check["LoanNumber"] == 2088118307, 'BorrowerState'] = 'SD'
fips_check.loc[fips_check["LoanNumber"] == 2088118307, 'Lat'] = 45.852921205907556
fips_check.loc[fips_check["LoanNumber"] == 2088118307, 'Long'] = -103.2527728723966
fips_check.loc[fips_check["LoanNumber"] == 2088118307, 'FIPS_z'] = '460639687001050'
fips_check.loc[fips_check["LoanNumber"] == 2088118307, 'full_add'] = "10199 Lanesboro Rd, Ludlow, SD"
fips_check.loc[fips_check["LoanNumber"] == 6173768404, 'Lat'] = 42.80066020690699
fips_check.loc[fips_check["LoanNumber"] == 6173768404, 'Long'] = -102.20017460322362
fips_check.loc[fips_check["LoanNumber"] == 6173768404, 'FIPS_z'] = '311619516001121'
fips_check.loc[fips_check["LoanNumber"] == 2678567709, 'BorrowerState'] = 'NY'
fips_check.loc[fips_check["LoanNumber"] == 4924757408, 'BorrowerState'] = 'MI'
fips_check.loc[fips_check["LoanNumber"] == 5226427204, 'Lat'] = 41.457798962489825
fips_check.loc[fips_check["LoanNumber"] == 5226427204, 'Long'] = -75.18055404561171
fips_check.loc[fips_check["LoanNumber"] == 5226427204, 'FIPS_z'] = '421039503031013'
'''


'\nfips_check.loc[fips_check["LoanNumber"] == 3506697210, \'BorrowerState\'] = \'IN\'\nfips_check.loc[fips_check["LoanNumber"] == \t1919668403, \'Lat\'] = 45.939522707297485\nfips_check.loc[fips_check["LoanNumber"] == \t1919668403, \'Long\'] = -86.28201139556197\nfips_check.loc[fips_check["LoanNumber"] == \t1919668403, \'FIPS_z\'] = \'261530002004003\'\nfips_check.loc[fips_check["LoanNumber"] == \t2703197109, \'Lat\'] = 45.939522707297485\nfips_check.loc[fips_check["LoanNumber"] == \t2703197109, \'Long\'] = -86.28201139556197\nfips_check.loc[fips_check["LoanNumber"] == \t2703197109, \'FIPS_z\'] = \'261530002004003\'\nfips_check.loc[fips_check["LoanNumber"] ==  4357417000, \'FIPS_z\'] = \'170850202004000\'\nfips_check.loc[fips_check["LoanNumber"] == \t4139637205, \'Lat\'] = 32.99734644174747\nfips_check.loc[fips_check["LoanNumber"] == \t4139637205, \'Long\'] = -93.46100681895895\nfips_check.loc[fips_check["LoanNumber"] == \t4139637205, \'FIPS_z\'] = \'221190313003012\'\nfips_check.loc[f

<b>Check BorrowerState Matches full_add ST<br>

Strip Leading / Trailing whitespaces from full_add column

In [120]:
fips_check["full_add"]=fips_check["full_add"].str.strip()

In [121]:
full_ch = fips_check.loc[(fips_check["BorrowerState"].values) != (fips_check["full_add"].str[-2:].values)]
full_ch

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
5248,9072157102,MS,"450 State Line Rd, Wilmer, AL",30.675827,-88.412523,280590401062013
7532,6549927102,MS,"450 State Line Rd, Wilmer, AL",30.675582,-88.412486,280590401062013
8634,7620227102,MS,"450 STATE LINE RD S, WILMER, AL",30.675827,-88.412523,280590401062013
520429,5657827001,ND,"16132 MT-200, Fairview, MT",47.861405,-104.040129,380539625002060
587764,2678567709,NY,"538 Lakeside Rd, Hewitt, NJ",41.185849,-74.330957,360710148013010
708743,4924757408,MI,"6205 N Summit St, Toledo, OH",41.732398,-83.480558,261158337003055
709627,3506697210,IN,"10001 N State Line Rd, Union City, OH",40.208960,-84.805814,181359516001007


DF - Adjusted States not matching (BorrowerState != FIPS -2:) occuring on State Lines <br>
9072157102, 6549927102, 7620227102, 3506697210, 5657827001, 2678567709, 4924757408 are examples of this.

In [122]:
full_ch_notna = full_ch.loc[full_ch["BorrowerState"]!="Not Available"]
full_ch_notna

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
5248,9072157102,MS,"450 State Line Rd, Wilmer, AL",30.675827,-88.412523,280590401062013
7532,6549927102,MS,"450 State Line Rd, Wilmer, AL",30.675582,-88.412486,280590401062013
8634,7620227102,MS,"450 STATE LINE RD S, WILMER, AL",30.675827,-88.412523,280590401062013
520429,5657827001,ND,"16132 MT-200, Fairview, MT",47.861405,-104.040129,380539625002060
587764,2678567709,NY,"538 Lakeside Rd, Hewitt, NJ",41.185849,-74.330957,360710148013010
708743,4924757408,MI,"6205 N Summit St, Toledo, OH",41.732398,-83.480558,261158337003055
709627,3506697210,IN,"10001 N State Line Rd, Union City, OH",40.208960,-84.805814,181359516001007


In [59]:
#fips_check.loc[fips_check["LoanNumber"] == 2910307108, 'full_add'] = '500 N 21st St, Monroe, LA'